# Assignment 3: SQL and Pandas

## Overview

Like Assignment 2, we will be using the Chinook database, this time focusing more on the sales side of the music store.

### Question 0

Set up your connection and cursor for `sqlite3`.

In [9]:
import sqlite3
import pandas as pd
from pandasql import sqldf

connection = sqlite3.connect("../data/chinook.sqlite")
cursor = connection.cursor()

### Question 1

Using `pandas`' built-in function `read_sql_query(query, connection)`, query the entire Customer and Invoice tables, merging by CustomerId.

In [10]:
customer_invoice_df = pd.read_sql_query('''
SELECT * FROM Customer
INNER JOIN Invoice
    ON Customer.CustomerId = Invoice.CustomerId
''', connection)

customer_invoice_df.head()

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,...,SupportRepId,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,...,5,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,...,4,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,...,4,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,...,5,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,23,John,Gordon,None,69 Salem Street,Boston,MA,USA,2113,+1 (617) 522-1333,...,4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


### Question 2

Using sqldf, query a new DataFrame containing only the CustomerId, FirstName, LastName, InvoiceDate, and Total. **Drop any invoices before 2010**. Remember, when using an existing pandas DataFrame, you're query FROM the variable name, not the tables from previously.  

In [28]:
table = sqldf('''
    SELECT CustomerId, FirstName, LastName, InvoiceDate, Total
    FROM customer_invoice_df
    WHERE InvoiceDate > 2010
''')

table

,CustomerId,FirstName,LastName,InvoiceDate,Total
0,43,Isabelle,Mercier,2010-01-08 00:00:00,1.98
1,45,Ladislav,Kovács,2010-01-08 00:00:00,1.98
2,47,Lucas,Mancini,2010-01-09 00:00:00,3.96
3,51,Joakim,Johansson,2010-01-10 00:00:00,6.94
4,57,Luis,Rojas,2010-01-13 00:00:00,17.91
...,...,...,...,...,...
324,25,Victor,Stevens,2013-12-05 00:00:00,3.96
325,29,Robert,Brown,2013-12-06 00:00:00,5.94
326,35,Madalena,Sampaio,2013-12-09 00:00:00,8.91
327,44,Terhi,Hämäläinen,2013-12-14 00:00:00,13.86


## Question 3

We now have a simplified list of invoices containing only name, id, date, and total. Let's figure out how much every has spent since 2010! Recall that pandas has the `groupby()` function (documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)). Using this, make a new DataFrame grouping by the CustomerId, giving the total/sum for each. 

In [29]:
table.groupby(["CustomerId"]).sum()

,Total
CustomerId,
1,39.62
2,12.87
3,39.62
4,28.73
5,38.64
6,40.71
7,40.64
8,30.69
9,31.68


### Question 4

Turns out, the pandas `groupby()` statement has an equivalent in SQL! Putting 

`GROUP BY <Table Name>.<Column Name>`

at the end of a query plus the keyword `SUM()` around the column you're grouping by creates a very similar table to the one we made in pandas!

Use what we've learned to query a table with FirstName, LastName, and SUM(Total). Do the results match what we did in pandas?

In [33]:
table = sqldf('''
    SELECT FirstName, LastName, SUM(Total)
    FROM customer_invoice_df
    WHERE InvoiceDate > 2010
    GROUP BY CustomerId
''')

table

,FirstName,LastName,SUM(Total)
0,Luís,Gonçalves,39.62
1,Leonie,Köhler,12.87
2,François,Tremblay,39.62
3,Bjørn,Hansen,28.73
4,František,Wichterlová,38.64
5,Helena,Holý,40.71
6,Astrid,Gruber,40.64
7,Daan,Peeters,30.69
8,Kara,Nielsen,31.68
9,Eduardo,Martins,28.71
